In [1]:
# для того, чтобы увеличить максииальный объём файла, который можно загружать в память
import sys
import csv
def find_and_set_sys_maxsize():
    maxInt = sys.maxsize
    decrement = True
    while decrement:
        decrement = False
        try:
            csv.field_size_limit(maxInt)
        except OverflowError:
            maxInt = int(maxInt/10)
            decrement = True
find_and_set_sys_maxsize()

# отключим ворнинги
import warnings
def fxn():
    warnings.warn("deprecated", DeprecationWarning)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

import os
import pandas as pd
import numpy as np

import gensim

# штука для предобработки текста
import nltk
nltk.download('punkt')
# from nltk.tokenize import word_tokenize

import pickle
import math
import datetime
def timestamp(): return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from scipy.sparse import hstack

import xgboost as xgb

[nltk_data] Downloading package punkt to C:\Users\Black
[nltk_data]     Overlord\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
granddebat_folder = '../GrandDébat/granddebat.fr/'
democrat_path = os.path.join(granddebat_folder, 'DEMOCRATIE_ET_CITOYENNETE.csv')
events_path = os.path.join(granddebat_folder, 'EVENTS.csv')
fiscalit_path = os.path.join(granddebat_folder, 'LA_FISCALITE_ET_LES_DEPENSES_PUBLIQUES.csv')
ecolog_path = os.path.join(granddebat_folder, 'LA_TRANSITION_ECOLOGIQUE.csv')
organisat_path = os.path.join(granddebat_folder, 'ORGANISATION_DE_LETAT_ET_DES_SERVICES_PUBLICS.csv')

french_encoding = "utf-8"

In [3]:
file_path = democrat_path

In [4]:
db_raw = pd.read_csv(file_path, engine='python', encoding=french_encoding)

In [5]:
questions_reverse_index = 37
db_questions = db_raw.loc[:, db_raw.columns[-questions_reverse_index:]]

# (QA4893whateverA3, QA4893whateverA4) -> (A3, A4)
question_list = db_raw.columns[-questions_reverse_index:]
question_list = [el.split()[0][-2:] for el in question_list]
db_questions.columns = question_list

db_questions.head()

,A3,A4,A5,Ew,Ex,Ey,Ez,E0,E1,E2,...,M1,M2,M3,M4,M5,Qx,Qy,Qz,Q0,Q1
0,Le citoyen,Non,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afin d’éviter de creuser les inégalités ne plu...
1,Un instrument de démocratie locale à modernise...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nous proposons le retour à la limitation de vi...
3,Voir l'intégralité de la proposition dans la d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POUR UN NOUVEAU CONTRAT CITOYEN ...
4,"député, maire, moi même",Non,NaN,"Budget participatif, possibilité d'interpeller...",Une bonne chose,NaN,NaN,Oui,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
region_names_to_zipCode_regions = {
"Île de France": ["75", "77", "78", "91", "92", "93", "94", "95"],
"Nord-Pas-de-Calais Picardie": ["02", "59", "60", "62", "80"],
"Alsace Champagne-Ardenne Lorraine": ["08", "10", "51", "52", "54", "55", "57", "67", "88", "68"],
"Normandie": ["14", "27", "50", "61", "76"],
"Bretagne": ["22", "29", "35", "56"],
"Pays de la Loire": ["44", "49", "53", "72", "85"],
"Centre Val de Loire": ["18", "28", "36", "37", "41", "45"],
"Bourgogne Franche-Comté": ["21", "25", "39", "58", "70", "71", "89", "90"],
"Aquitaine Limousin Poitou-Charentes": ["16", "17", "19", "23", "24", "33", "40", "47", "64", "79", "86", "87"],
"Auvergne-Rhône-Alpes": ["01", "03", "07", "15", "26", "38", "42", "43", "63", "69", "73", "74"],
"Languedoc-Roussillon Midi Pyrénées": ["09", "11", "12", "30", "31", "32", "34", "46", "48", "65", "66", "81", "82"],
"Provence Alpes Côte d'Azur": ["04", "05", "06", "13", "83", "84"],
"Corse": ["20"],
}

In [7]:
def preprocess_text(text):
    text = " ".join(gensim.utils.simple_preprocess(text))
    return text

def strip_zipCode(zipCode):
    return str(zipCode)[0:2]

def get_region_by_stripped_zipCode(zipCode):
    region_index = None
    for idx, region in enumerate(region_names_to_zipCode_regions):
        if zipCode in region_names_to_zipCode_regions[region]:
            region_index = idx
    return region_index

def preprocess_zipCode(zipCode):
    zipCode = strip_zipCode(zipCode)
    return get_region_by_stripped_zipCode(zipCode)

In [8]:
%%time

db = db_questions

texts = []
authorZipCode_regions = []
# question_numbers = []
i = 0
invalid_zipcodes = []

for row in range(db.shape[0]):
    if row % 10000 == 0: print("row", row, "/", db.shape[0])
    for col in range(db.shape[1]):
        if not pd.isnull(db.iat[row, col]):
            text = preprocess_text(db.iat[row, col])
            authorZipCode = db_raw.at[row, 'authorZipCode']
            authorZipCode_region = preprocess_zipCode(authorZipCode)
#             question_number = col
            
            if authorZipCode_region == None:
                invalid_zipcodes.append(strip_zipCode(authorZipCode))
                authorZipCode_region = -1

            texts.append(text)
            authorZipCode_regions.append(authorZipCode_region)
#             question_numbers.append(question_number)

invalid_zipcodes = set(invalid_zipcodes)
print("zipCodes that does not belong to any region:", invalid_zipcodes)

row 0 / 65758
row 10000 / 65758
row 20000 / 65758
row 30000 / 65758
row 40000 / 65758
row 50000 / 65758
row 60000 / 65758
zipCodes that does not belong to any region: {'9', '96', '97', '4', '7', '99', '5', '1', '6', '0', '98', '3'}
Wall time: 2min 27s


In [9]:
for idx, zipCode in enumerate(authorZipCode_regions):
    if zipCode == -1:
        del texts[idx]
        del authorZipCode_regions[idx]

#### On this step we have a [texts] and [authorZipCode_regions] lists
The first one has strings of answers as elements  
The second one has one of ~14 regions

In [10]:
def join_text_with_same_tag(texts, tags):
    new_texts = []
    new_tags = []
    tag_to_text = {}
    for tag in set(tags):
        tag_to_text[tag] = []
    
    for idx, tag in enumerate(tags):
        if idx % (len(tags) // 10) == 0: print("%.0f%% done" % (idx / len(tags) * 100))
        tag_to_text[tag].append(texts[idx])
    for tag in tag_to_text:
        tag_to_text[tag] = " ".join(tag_to_text[tag])
    for tag in tag_to_text:
        new_texts.append(tag_to_text[tag])
        new_tags.append(tag)
    return new_texts, new_tags

In [11]:
all_region_texts, regions = join_text_with_same_tag(texts, authorZipCode_regions)

0% done
10% done
20% done
30% done
40% done
50% done
60% done
70% done
80% done
90% done
100% done


In [12]:
len(all_region_texts)

14

In [13]:
print(regions)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, -1]


#### On this step we have a [all_region_texts] and [regions] lists
The first one has string of all the answers concatenacted for one region as the element
The second one has a label for each region

In [14]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [15]:
corpus = all_region_texts

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

cv=CountVectorizer(max_df=0.25)
word_count_vector=cv.fit_transform(corpus)

In [17]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

#### On this step what we already have is: we have counted the frequency of words in each region. We just have the words-region frequency matrix now

##### The region index (corpus_idx). If you want to see the most important words of the region 1, do corpus_idx = 1

In [18]:
corpus_idx = 1
answer = corpus[corpus_idx]

In [19]:
def get_words_to_weights_for_region_dict(answer, vocabulary_limit = 10):
    words_to_weights_for_region_dict = {}
    tf_idf_vector=tfidf_transformer.transform(cv.transform([answer]))
    print(tf_idf_vector)
    feature_names=cv.get_feature_names()
    sorted_items=sort_coo(tf_idf_vector.tocoo())
    keywords=extract_topn_from_vector(feature_names,sorted_items, vocabulary_limit)
    

    for idx, el in enumerate(keywords):
        keywords[el] = keywords[el] / (idx + 10)

    norm = keywords[list(keywords)[0]]
    for idx, el in enumerate(keywords):
        keywords[el] /= norm
        words_to_weights_for_region_dict[el] = keywords[el]
    return words_to_weights_for_region_dict

#### The problem is that tfidf was trained using large large string as a corpus  
#### But now it does not want to vectorize sentences less than N in length, so WE DON'T USE TF-IDF WEIGHTING

In [20]:
print(get_words_to_weights_for_region_dict(answer[:20000]))

  (0, 79560)	0.2520009270350179
  (0, 54857)	0.3272343900309138
  (0, 38897)	0.3272343900309138
  (0, 35695)	0.283225635378922
  (0, 32533)	0.3272343900309138
  (0, 31865)	0.283225635378922
  (0, 31864)	0.283225635378922
  (0, 31614)	0.283225635378922
  (0, 28416)	0.3272343900309138
  (0, 18568)	0.3272343900309138
  (0, 11249)	0.283225635378922
{'pléthoriennes': 1.0, 'inititative': 0.9090909090909091, 'frime': 0.8333333333333334, 'espacce': 0.7692307692307693, 'defiscalisant': 0.7142857142857143, 'histogrammes': 0.5769622833843017, 'fonderies': 0.5409021406727829, 'fonderie': 0.5090843676920309, 'flu': 0.4808019028202514, 'certianes': 0.4554965395139223}


In [21]:
print(get_words_to_weights_for_region_dict(answer[:2000]))

IndexError: list index out of range

##### So what we have already, is the most important words for the region with idx == 1

### We have answers in X, regions in y  
### We have weights that will say which words are more important in the answer  
### So let's get the vector for each word of the answer

In [22]:
from gensim.models.word2vec import Word2Vec
w2v_model = Word2Vec.load("./pretrained_w2v/fr.bin")
# https://github.com/Kyubyong/wordvectors
# French (w)

#### You can have some fun with the pre-trained w2vec model

In [23]:
w2v_model.most_similar('écosystème')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('biotope', 0.6826141476631165),
 ('environnement', 0.6822277903556824),
 ('aquifère', 0.6639423966407776),
 ("l'écosystème", 0.6581449508666992),
 ('biome', 0.6514424085617065),
 ('microclimat', 0.6483206152915955),
 ('habitat', 0.6472629904747009),
 ('humus', 0.616238534450531),
 ('écosystèmes', 0.597100019454956),
 ('anticyclone', 0.5882144570350647)]

#### We can get the vector of the word (10 / 300 components displayed here)

In [24]:
w2v_model['écosystème'][0:10]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.6136349 ,  1.5202744 , -1.5171983 ,  1.0266268 , -0.12849323,
        0.8902757 ,  0.47812438, -0.56151843, -1.2719674 ,  0.8245529 ],
      dtype=float32)

### Here we convert an individual answer sentences to vectors using word2vec pretrained model

In [25]:
texts[0]

'le citoyen'

In [ ]:
# w2vec_texts_vectors = np.array((len(texts), 1))
# unknown_to_pretrained_word_list = []
# for idx, text in enumerate(texts):
#     final_idx = len(texts)
#     if idx % (final_idx // 10) == 0: print("%.0f%% done" % (idx / final_idx * 100))
#     word_vectors = []
# #     word_weights = []
# #     text_words_weights = get_words_to_weights_for_region_dict(text)
#     for word in text.split():
# #         if word in text_words_weights:
#         try:
#             word_vectors.append(w2v_model[word])
#         except:
#             if not word in unknown_to_pretrained_word_list:
#                 unknown_to_pretrained_word_list.append(word)
# #             word_weights.append(text_words_weights[word])
#     if not len(word_vectors) == 0:
#         word_vectors = np.array(word_vectors)
#     else:
#         word_vectors = np.zeros((300,))
# #     word_weights = np.array(word_weights)
# #     word_weighted_vectors = word_vectors * word_weights[:, np.newaxis]
# #     word_combined_vector = word_weighted_vectors.sum(axis = 0) / word_weighted_vectors.shape[0]
#     words_combined_vector = word_vectors.sum(axis = 0) / word_vectors.shape[0]
# #     w2vec_texts_vectors.append(words_combined_vector)
#     w2vec_texts_vectors.append(words_combined_vector)

#### The procedure below just goes through all the answers and averages w2vec vectors for answer sentence (it is not so bad, since the dimensionality of pre-trained model is very high, so much of the information will be conserved

In [51]:
w2vec_texts_vectors = []
unknown_to_pretrained_word_list = []
for idx, text in enumerate(texts):
    final_idx = len(texts)
    if idx % (final_idx // 10) == 0: print("%.0f%% done" % (idx / final_idx * 100))
    word_vectors = []
    for word in text.split():
        try:
            word_vectors.append(w2v_model[word])
        except:
            if not word in unknown_to_pretrained_word_list:
                unknown_to_pretrained_word_list.append(word)
    if not len(word_vectors) == 0:
        word_vectors = np.array(word_vectors, dtype=np.float16)
        words_combined_vector = word_vectors.sum(axis = 0) / word_vectors.shape[0]
    else:
        words_combined_vector = np.zeros((300,), dtype=np.float16)
    w2vec_texts_vectors.append(words_combined_vector)

0% done


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


10% done
20% done
30% done
40% done
50% done
60% done
70% done
80% done
90% done
100% done


In [ ]:
unknown_to_pretrained_word_list

In [73]:
w2vec_texts_vectors_np = np.array(w2vec_texts_vectors, dtype=np.float16)#.reshape(-1, 1)
# w2vec_texts_vectors_np = np.vstack(w2vec_texts_vectors)#.reshape(-1, 1)

In [79]:
w2vec_texts_vectors_np -= w2vec_texts_vectors_np.min()
w2vec_texts_vectors_np /= w2vec_texts_vectors_np.max()

In [81]:
w2vec_texts_vectors_np.shape

(1590723, 300)

#### For now, we've got the list w2vec_texts_vectors that has a vector of each answer sentence as it's element  
#### Now we can fit it to the algorithm 

In [55]:
X_train = w2vec_texts_vectors_np
y_train = authorZipCode_regions

### XGBoost

In [ ]:
dtrain = xgb.DMatrix(X_train, label = y_train)

In [ ]:
param = {}
# param['silent'] = 1
# param['verbosity'] = 2
param['nthread'] = 4

param['max_depth'] = 6
num_boost_round = 10


bst = xgb.train(param, dtrain, num_boost_round = num_boost_round)

In [ ]:
xgb_X_test = xgb.DMatrix(X_test)
xgb_y_test = xgb.DMatrix(y_test)

y_pred = bst.predict(xgb_X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Model accuracy: %.2f%%" % (accuracy * 100.0))
random_accuracy = 1 / len(set(authorZipCode_regions))
print("Random accuracy: %.2f%%" % (random_accuracy * 100.0))

In [80]:
from sklearn.naive_bayes import MultinomialNB as NB

# fit_prior = False decreases acc by 0.1% which means that we have a slight imbalance between zipcodes
nb_model = NB().fit(X=X_train, y=y_train)

MemoryError: 

#### Go buy some RAM :>
#### I get memory errors on both of the models

In [ ]:
y_pred = nb_model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Model accuracy: %.2f%%" % (accuracy * 100.0))
random_accuracy = 1 / len(set(authorZipCode_regions))
print("Random accuracy: %.2f%%" % (random_accuracy * 100.0))